In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("RetailDataAnalysis").getOrCreate()

staticDataFrame = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("/content/sample_data/retail_data/*.csv")

staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema


In [9]:
from pprint import pprint,pformat


pformat(staticSchema)

"StructType([StructField('InvoiceNo', StringType(), True), StructField('StockCode', StringType(), True), StructField('Description', StringType(), True), StructField('Quantity', IntegerType(), True), StructField('InvoiceDate', TimestampType(), True), StructField('UnitPrice', DoubleType(), True), StructField('CustomerID', DoubleType(), True), StructField('Country', StringType(), True)])"

In [11]:
 spark.conf.set("spark.sql.shuffle.partitions", "5")

In [ ]:
# we are working with time–series data
#  let’s  add a total cost column and see on what days a customer spent the most
# The window function will include all data from each day in the aggregation. It’s simply a
# window over the time–series column in our data

In [12]:
from pyspark.sql.functions import window, column, desc, col

staticDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
.groupBy(
 col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
 .sum("total_cost")\
 .show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   14075.0|{2011-12-05 00:00...|316.78000000000003|
|   18180.0|{2011-12-05 00:00...|            310.73|
|   15358.0|{2011-12-05 00:00...| 830.0600000000003|
|   15392.0|{2011-12-05 00:00...|304.40999999999997|
|   15290.0|{2011-12-05 00:00...|263.02000000000004|
+----------+--------------------+------------------+
only showing top 5 rows


**Take a look at the streaming code**


*   Biggest change is that we used readStream instead of read
*   MaxFilesPerTrigger option, which simply specifies  the number of files we  should read in at once




In [13]:
streamingDataFrame = spark.readStream\
 .schema(staticSchema)\
 .option("maxFilesPerTrigger", 1)\
    .format("csv")\
    .option("header", "true")\
    .load("/content/sample_data/retail_data/*.csv")

In [15]:
streamingDataFrame.isStreaming

True

In [16]:
purchaseByCustomerPerHour = streamingDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")

In [ ]:
# will output to an in-memory table that
 #we will update after each trigger

In [17]:
purchaseByCustomerPerHour.writeStream\
    .format("memory")\
    .queryName("customer_purchases")\
    .outputMode("complete")\
    .start()

In [21]:
spark.sql("""
  SELECT *
  FROM customer_purchases
  ORDER BY `sum(total_cost)` DESC
  """)\
  .show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|      NULL|{2011-12-08 00:00...|31975.590000000007|
|      NULL|{2010-12-21 00:00...|31347.479999999938|
|   18102.0|{2010-12-07 00:00...|          25920.37|
|      NULL|{2010-12-10 00:00...|25399.560000000012|
|      NULL|{2010-12-17 00:00...|25371.769999999768|
+----------+--------------------+------------------+
only showing top 5 rows


In [24]:
purchaseByCustomerPerHour.writeStream\
.format("console")\
.queryName("customer_purchases_2")\
.outputMode("complete")\
.start()